# Model evaluation for the Dengue Serotype Classifier

The aim of this notebook is to test different classification algorithms for Dengue serotypes from genomic sequence fragments 

## Preprocessing labeled data

A second approach involves processing _fasta_ files for each serotype.

This time, we use the biopython module to import data from files with fasta format. 

In [53]:
from Bio import SeqIO
import pandas as pd

In [54]:
fasta_files = [['../data/datos_entrenamiento/D' + str(i) + '_protE_aligned.fasta', str(i)] for i in range(1,5)]

In [55]:
fasta_files

[['../data/datos_entrenamiento/D1_protE_aligned.fasta', '1'],
 ['../data/datos_entrenamiento/D2_protE_aligned.fasta', '2'],
 ['../data/datos_entrenamiento/D3_protE_aligned.fasta', '3'],
 ['../data/datos_entrenamiento/D4_protE_aligned.fasta', '4']]

In [56]:
def load_fasta_files(files= []):
    '''
    Load sequences from fasta files
    
    -----
    param:
    files list containing [path, label]
    
    -------
    returns:
    list containing [label, id, sequence]
    '''
    res = []
    for f in files:
        fasta_file = SeqIO.parse(open(f[0]), 'fasta')
        
        for s in fasta_file:
            res.append([s.id, str(s.seq), f[1]])
    
    return res

Let's load the data into a pandas.DataFrame

In [57]:
data = load_fasta_files(fasta_files)

df = pd.DataFrame(data, columns=['id', 'sequence', 'label'])

In [58]:
df.shape

(10103, 3)

In [59]:
len(df.iloc[0]['sequence'])

1898

## Feature extraction

In [60]:
import sys

sys.path.append('../utils/')

import util

In [61]:
df['sequence'].shape

(10103,)

In [62]:
result = {}

#Replace every character not included in [actg-] with '-'
data_sequences = df['sequence'].replace(to_replace=r'(?![actg\-]).', value='-', regex=True)

for idx, data in data_sequences.iteritems():
    splitted_string = util.insert_separator(data).split(',')
        
    result[idx] = splitted_string

In [63]:
df_sequences = pd.DataFrame.from_dict(result, orient='index')

In [64]:
assert(len(result) == df_sequences.shape[0]) #4805

In [65]:
df_sequences.shape

(10103, 1898)

In [66]:
from sklearn.preprocessing import OneHotEncoder

In [67]:
enc = OneHotEncoder() #(['-', 'a', 'c', 't', 'g'])

df_sequences_dummies = enc.fit_transform(df_sequences)

In [68]:
df_sequences_dummies.shape

(10103, 7086)

We can take a look at the feature names.

In [69]:
enc.get_feature_names()

array(['x0_-', 'x0_g', 'x1_-', ..., 'x1896_c', 'x1897_-', 'x1897_c'],
      dtype=object)

## Feature selection

Let's remove features with zero variance using scikit learn

In [70]:
from sklearn.feature_selection import VarianceThreshold

In [71]:
sel = VarianceThreshold()

In [72]:
sequences_reduced = sel.fit_transform(df_sequences_dummies)

In [73]:
sequences_reduced.shape

(10103, 7076)

In [74]:
sel.get_support().shape

(7086,)

By removing features with zero variability, we demonstrate that there is no significant dimensionality reduction. Only 4 of the 7439 present zero variability.

## Model training

Now that we have a feature matrix, we can train a model.  

But first, split the data set into test and train.

In [75]:
y = df['label']

In [76]:
y.shape

(10103,)

In [77]:
from sklearn.model_selection import train_test_split

In [78]:
X_train, X_test, y_train, y_test = train_test_split(sequences_reduced, y, test_size=0.4, random_state=0)

In [79]:
X_train.shape, y_train.shape

((6061, 7076), (6061,))

In [80]:
X_test.shape, y_test.shape

((4042, 7076), (4042,))

We try different algorithms for multiclass classification, including

* BernoulliNB
* Decision Trees
* SVM
* Deep learning

### Naive Bayes

In [81]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

from sklearn.naive_bayes import BernoulliNB

In [82]:
clf = BernoulliNB()

In [83]:
clf.fit(X_train, y_train)

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

In [84]:
y_pred = clf.predict(X_test)

In [85]:
accuracy_score(y_test, y_pred)

0.9990103908955963

In [86]:
confusion_matrix(y_test, y_pred)

array([[1908,    4,    0,    0],
       [   0, 1106,    0,    0],
       [   0,    0,  518,    0],
       [   0,    0,    0,  506]])

### Decision trees

In [87]:
from sklearn.tree import DecisionTreeClassifier

In [88]:
clf = DecisionTreeClassifier()

In [89]:
clf.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [90]:
y_pred = clf.predict(X_test)

In [91]:
accuracy_score(y_test, y_pred)

0.9982681840672935

In [92]:
confusion_matrix(y_test, y_pred)

array([[1908,    4,    0,    0],
       [   2, 1104,    0,    0],
       [   0,    0,  518,    0],
       [   1,    0,    0,  505]])

### SVM

In [93]:
from sklearn.svm import LinearSVC

In [94]:
clf = LinearSVC(multi_class="crammer_singer")

In [95]:
clf.fit(X_train, y_train)

/home/zamu/.virtualenvs/dengue/lib/python3.5/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='crammer_singer', penalty='l2', random_state=None,
          tol=0.0001, verbose=0)

In [96]:
y_pred = clf.predict(X_test)

In [97]:
accuracy_score(y_test, y_pred)

1.0

In [98]:
confusion_matrix(y_test, y_pred)

array([[1912,    0,    0,    0],
       [   0, 1106,    0,    0],
       [   0,    0,  518,    0],
       [   0,    0,    0,  506]])

### Neural network

In [99]:
from sklearn.neural_network import MLPClassifier

In [100]:
clf = MLPClassifier()

In [101]:
clf.fit(X_train, y_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100,), learning_rate='constant',
              learning_rate_init=0.001, max_iter=200, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=None, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [102]:
y_pred = clf.predict(X_test)

In [103]:
accuracy_score(y_test, y_pred)

1.0

In [104]:
confusion_matrix(y_test, y_pred)

array([[1912,    0,    0,    0],
       [   0, 1106,    0,    0],
       [   0,    0,  518,    0],
       [   0,    0,    0,  506]])

## Conclusion

The classification task for dengue serotypes reach an accuracy score of 1.0 for SVM and MLP algorithms. 
Feature engineering involves the creation of dummy variables and zero variability features removal.